In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Função para pré-processar os dados
def preprocess_data(df):
    # Preencher valores ausentes em 'num_fotos' e 'veiculo_alienado'
    df['num_fotos'].fillna(0, inplace=True)
    df['veiculo_alienado'].fillna(0, inplace=True)

    # Converter valores categóricos para binários
    df['dono_aceita_troca'] = np.where(df['dono_aceita_troca'] == 'Aceita troca', 1, 0)
    df['veiculo_único_dono'] = np.where(df['veiculo_único_dono'] == 'Único dono', 1, 0)
    df['revisoes_concessionaria'] = np.where(df['revisoes_concessionaria'] == 'Todas as revisões feitas pela concessionária', 1, 0)
    df['ipva_pago'] = np.where(df['ipva_pago'] == 'IPVA pago', 1, 0)
    df['veiculo_licenciado'] = np.where(df['veiculo_licenciado'] == 'Licenciado', 1, 0)
    df['garantia_de_fábrica'] = np.where(df['garantia_de_fábrica'] == 'Garantia de fábrica', 1, 0)
    df['revisoes_dentro_agenda'] = np.where(df['revisoes_dentro_agenda'] == 'Todas as revisões feitas pela agenda do carro', 1, 0)
    
    # Converter colunas categóricas para o tipo string
    categorical_columns = ['marca', 'modelo', 'versao', 'cambio', 'tipo', 'cor', 'tipo_vendedor',
                           'cidade_vendedor', 'estado_vendedor', 'anunciante']
    df[categorical_columns] = df[categorical_columns].astype(str)
    
    # Criar as novas variáveis
    df['idade_veiculo'] = df['ano_modelo'] - df['ano_de_fabricacao']
    df['km_por_ano'] = df['hodometro'] / df['idade_veiculo']
    df['e_flex'] = df['versao'].str.contains('flex', case=False, na=False).astype(int)
    df['cambio_automatico'] = df['versao'].str.contains('automatico|automático', case=False, na=False).astype(int)
    
    # Remover colunas 'veiculo_alienado' e 'veiculo_alienado'
    df.drop(['veiculo_alienado', 'veiculo_alienado'], axis=1, inplace=True)
    df['idade_veiculo'] = df['idade_veiculo'].clip(0, 10)  # Limitar a idade do veículo entre 0 e 10 anos
    df['km_por_ano'].replace([np.inf, -np.inf], np.nan, inplace=True)  # Substituir infinito por NaN
    df['km_por_ano'].fillna(df['hodometro'], inplace=True)  # Preencher valores NaN com hodômetro (caso tenha havido divisão por zero)
    
    return df

In [3]:
# Carregar os dados de treinamento e teste a partir de arquivos Excel
df_treino = pd.read_excel("C:/Users/cgdb2/Downloads/treino_carros.xls")
df_teste = pd.read_excel("C:/Users/cgdb2/Downloads/teste_carros.xls")

In [4]:
# Pré-processar os dados de treinamento e teste
df_treino = preprocess_data(df_treino)
df_teste = preprocess_data(df_teste)

combined_data = pd.concat([df_treino.drop('preco', axis=1), df_teste], ignore_index=True)

In [5]:
# Codificar as variáveis categóricas usando One-Hot Encoding com todas as colunas categóricas presentes em ambos os conjuntos
combined_encoded = pd.get_dummies(combined_data)

In [6]:
# Separar novamente os conjuntos de treinamento e teste após a codificação
X_train_encoded = combined_encoded.iloc[:len(df_treino)]
X_test_encoded = combined_encoded.iloc[len(df_treino):]

In [7]:
# 1. Normalização das variáveis contínuas
scaler = StandardScaler()
numerical_columns = ['num_fotos', 'ano_de_fabricacao', 'ano_modelo', 'hodometro', 'num_portas', 'idade_veiculo', 'km_por_ano']
X_train_encoded[numerical_columns] = scaler.fit_transform(X_train_encoded[numerical_columns])
X_test_encoded[numerical_columns] = scaler.transform(X_test_encoded[numerical_columns])

In [8]:
# 2. Regularização com Ridge
ridge_model = Ridge(alpha=1.0)  # Alpha é o hiperparâmetro de regularização, você pode ajustá-lo conforme necessário
y_train = df_treino['preco']
y_pred_train_ridge = cross_val_predict(ridge_model, X_train_encoded, y_train, cv=5)
mse_train_ridge = mean_squared_error(y_train, y_pred_train_ridge)
r2_train_ridge = r2_score(y_train, y_pred_train_ridge)
mae_train_ridge = mean_absolute_error(y_train, y_pred_train_ridge)

In [9]:
print("Métricas com Ridge:")
print(f"Mean Squared Error (Train): {mse_train_ridge}")
print(f"R² Score (Train): {r2_train_ridge}")
print(f"Mean Absolute Error (Train): {mae_train_ridge}")


Métricas com Ridge:
Mean Squared Error (Train): 1855788934.188533
R² Score (Train): 0.7217123333903106
Mean Absolute Error (Train): 27542.009517370865


In [10]:
# 3. Seleção de Features com Lasso
lasso_model = Lasso(alpha=0.01)  # Alpha é o hiperparâmetro de regularização L1, você pode ajustá-lo conforme necessário
y_pred_train_lasso = cross_val_predict(lasso_model, X_train_encoded, y_train, cv=5)
mse_train_lasso = mean_squared_error(y_train, y_pred_train_lasso)
r2_train_lasso = r2_score(y_train, y_pred_train_lasso)
mae_train_lasso = mean_absolute_error(y_train, y_pred_train_lasso)

In [11]:
print("Métricas com Lasso:")
print(f"Mean Squared Error (Train): {mse_train_lasso}")
print(f"R² Score (Train): {r2_train_lasso}")
print(f"Mean Absolute Error (Train): {mae_train_lasso}")

Métricas com Lasso:
Mean Squared Error (Train): 1941455437.4976058
R² Score (Train): 0.7088660819264192
Mean Absolute Error (Train): 27984.35247348117


In [12]:

if r2_train_ridge > r2_train_lasso:
    selected_model = ridge_model
else:
    selected_model = lasso_model

In [13]:
# Treinar o modelo selecionado usando todos os dados de treinamento para fins de teste final
selected_model.fit(X_train_encoded, y_train)

Ridge()

In [14]:
X_test = X_test_encoded  # No need to drop 'id' column as it is not in X_test_encoded
y_pred_test = selected_model.predict(X_test)

In [15]:
# Criar DataFrame com as previsões
df_submissao = pd.DataFrame({'id': df_teste['id'], 'preco': y_pred_test})

In [ ]:
df_submissao.to_csv("C:/Users/cgdb2/Desktop/LH_CD_CLAUDIO_GABRIEL_DUARTE_BRANDAO/predicted.csv", index=False